# Create an OQt project

This notebook performs the initial steps needed to create an OQt project. It can be use either as a standalone notebook or as a script using runipy. 

This notebook can be run as a script in this way:
```shell
ini_filename = 'project.ini' runipy.py project_create.py
```

This script will create in the folder `/Users/home/hazard_projects/` a subdirectory called `nowhere` and inside `nowhere` there will be two files. The first one - named `nowhere.oqmbtp` - is a [pickle](https://docs.python.org/2/library/pickle.html) file while the second one - named `completeness.hdf5` - is a [hdf5](https://www.hdfgroup.org/HDF5/) file which will be used to store catalogue completeness.

## Settings

In [ ]:
import os
import re
import sys
import h5py

from oqmbt.oqt_project import OQtProject, OQtModel
from configparser import ConfigParser

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

## Reading .ini file

In [ ]:
# By default we assume that this notebook is run from the command line, therefore we look at environment variables.
try: 
    ini_filename = os.environ.get('ini_file')
    assert ini_filename is not None
except:
    #ini_filename = './../../../project.ini'
    #ini_filename = '/Users/mpagani/Hazard_projects/south_america.ini'
    #ini_filename = '/Users/mpagani/Hazard_projects/south_america_sf.ini'
    #ini_filename = '/home/openquake/Hazard_projects/south_east_china.ini'
    #ini_filename = './../../../examples/South_east_china/south_east_china.ini'
    #ini_filename = '/home/openquake/GEM/model_building_tools_data/data/south_america/south_america.ini'
    #ini_filename = '/home/openquake/GEM/model_building_tools_data/data/china/south_east_china.ini'
    ini_filename = '/home/openquake/GEM/model_building_tools_data/data/china2016/china2016.ini'
assert os.path.exists(ini_filename)
print 'Reading project information from: \n', ini_filename

In [ ]:
config = ConfigParser(dict_type=AttrDict)
config.read(ini_filename)

project_dir = os.path.abspath(config._sections.project.directory)
project_name = config._sections.project.name

print 'Project directory : %s' % (project_dir)
print 'Project name      : %s' % (project_name)

### Clean the folder
The commands inside this cell will delete and recreate from scratch the folder that will contain the project. ALL THE FILES PREVIOUSLY STORED IN THIS FOLDER WILL BE DELETED!

## Create project
This cell creates an instance of a [OQtProject](./../../../doc/_build/html/project.html) which will be used to store all the information related with the construction of the source model.
    

In [ ]:
prj = OQtProject(project_name, project_dir)
project_filename = os.path.join(project_dir, prj._get_filename())
print project_filename
print project_dir

## Create completeness file, the earthquake rate file and the file associating earthquakes to faults
This cell creates a .hdf5 file which will be used to store the information related with the magnitude-time completeness of the catalogue.

In [ ]:
prj.compl_hdf5_filename = 'completeness.hdf5'
compl_hdf5_filename = os.path.join(project_dir, prj.compl_hdf5_filename)
print compl_hdf5_filename
fhdf5 = h5py.File(compl_hdf5_filename, 'a')
fhdf5.close()

In [ ]:
prj.eqk_rates_hdf5_filename = 'eqk_rates.hdf5'
eqk_rates_hdf5_filename = os.path.join(project_dir, prj.eqk_rates_hdf5_filename)
fhdf5 = h5py.File(eqk_rates_hdf5_filename, 'a')
fhdf5.close()

In [ ]:
prj.hypo_close_to_flts_hdf5_filename = 'hypo_close_to_faults.hdf5'
hypo_close_to_flts_hdf5_filename = os.path.join(project_dir, prj.hypo_close_to_flts_hdf5_filename)
fhdf5 = h5py.File(hypo_close_to_flts_hdf5_filename, 'a')
fhdf5.close()

## Create source model and add it to the project

In [ ]:
for key in config._sections.keys():
    if re.search('^model', key):
        inimodel = getattr(config._sections, key)
        model = OQtModel(model_id=inimodel.id, name=inimodel.name)
        for mkey in inimodel.keys():
            if not re.search('^__', mkey):
                if re.search('filename$', mkey):
                    path = os.path.relpath(os.path.abspath(getattr(inimodel, mkey)))
                    setattr(model, mkey, path)
                else:
                    setattr(model, mkey, getattr(inimodel, mkey))
        prj.add_model(model)
prj.active_model_id = 'model01'

## Save project 

In [ ]:
prj.save()
del prj

## Check: load model 

In [ ]:
print 'Project file:', project_filename

In [ ]:
prj = OQtProject.load_from_file(project_filename)
prj.get_info()
model = prj.models['model01']
model.get_info()